In [2]:
#uncomment to install packages
#conda install -c conda-forge selenium
#!pip install fake_useragent
#!pip install eventlet
#!pip install python-settings

In [3]:
from bs4 import BeautifulSoup
import requests
import time, os
import re
import random
from random import randint
import random
from time import sleep

import pandas as pd
from pymongo import MongoClient
from pprint import pprint
import json

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [5]:
from fake_useragent import UserAgent

In [6]:
#helper files
import get_proxies as proxies
import extractors

### Create Mongo collection for urls

In [7]:
client = MongoClient()

In [8]:
db = client['philadelphia_home_prices']

In [14]:
db.list_collection_names()

['prices_messy', 'listing_urls']

In [10]:
#uncomment to delete the collection
#db.drop_collection('listing_urls')

In [11]:
db.list_collection_names()

['prices_messy', 'listing_data', 'listing_urls']

In [15]:
col = db.create_collection('listing_urls')

CollectionInvalid: collection listing_urls already exists

In [ ]:
db.list_collection_names()

### scrape pages for listing urls

In [ ]:
url = 'https://www.realtor.com/'

In [ ]:
# 44 listings per page. Choose start and stop page
page_from = 26
page_to = 26

In [ ]:
col = db.get_collection('listing_urls')

In [ ]:
col.count()

In [ ]:
for i in range(page_from, page_to+1):
    #print(i) #prints page that is open in case bot is detected
    driver = webdriver.Chrome(chromedriver)
    driver.get(url + 'soldhomeprices/Philadelphia_PA/pg-' + str(i))
    
    sleep(random.uniform(60,120))
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    page = str(soup.find_all('div', class_='photo-wrap'))
    extensions = re.findall(r'[A-z]+-[A-z]+\/[0-9-A-z]+[^"]', page)
    
    if len(extensions) == 0:
        print('bot is caught on page {}'.format(i))
        break
        
    for i in extensions:
        page_url = url + i
        page = {'url': page_url}
        col.insert_one(page)
        
    sleep(random.uniform(1,3))
    
    driver.close()

In [17]:
col.count()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


0

### Pull urls from mongoDB into memory

In [ ]:
#pull urls from mongo db
cursor = db.listing_urls.find({})
listings = list(cursor)

In [ ]:
#convert dictionary values to list
listing_urls_list = []
for i in listings:
    url = i['url']
    listing_urls_list.append(url)

In [ ]:
#see first 5 in list
listing_urls_list[0:5]

### Get data from urls

#### new collection for listing data

In [16]:
#uncomment the code you need to use
db.create_collection('listing_data')
col = db.get_collection('listing_data')

In [ ]:
db.collection_names()

In [13]:
#uncomment to drop
#db.drop_collection('listing_data')

{'nIndexesWas': 1, 'ns': 'philadelphia_home_prices.listing_data', 'ok': 1.0}

In [ ]:
db.collection_names()

#### Rotate proxies

In [ ]:
#get list of proxies from free-proxy.net
proxies_address = proxies.proxy_list()

In [ ]:
listing_from = 13
listing_to = len(listing_urls_list)
count = 0
tested_proxies = []
bad_proxies = []
bad_url = []

In [ ]:
import sys
import concurrent.futures
from datetime import datetime
import eventlet
import settings

In [ ]:
pool = eventlet.GreenPool(settings.max_threads)
pile = eventlet.GreenPile(pool)

In [ ]:
datetime.now()

In [ ]:
proxies

In [ ]:
for i in range(listing_from, listing_to+1): 
    print(i, listing_urls_list[i])

    try:
        #PROXY = proxies[count].get_address()
        PROXY = proxies_address[count]
        webdriver.DesiredCapabilities.CHROME['proxy']={
                                                        "httpProxy":PROXY,
                                                       "ftpProxy":PROXY,
                                                       "sslProxy":PROXY,
                                                       "proxyType":"MANUAL",
                                                        }
        driver = webdriver.Chrome(chromedriver)
        driver.get(listing_urls_list[i])

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        listing_data = get_data(soup)
        good_proxies.append(PROXY)

    except:
        bad_proxies.append(PROXY)
        count += 1
        #driver.close()
        
    driver.close()
    
    #insert listing data into mongo database
    col.insert_one(listing_data)
    print(listing_data)
    count += 1
    print(count)

In [ ]:
threads = min(MAX_THREADS, len(story_urls))

with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    executor.map(download_url, story_urls)

In [ ]:
from extractors import get_data 

In [ ]:
count

In [ ]:
for i in range(listing_from, listing_to+1): 
    print(i, listing_urls_list[i])
    check = False
    state = False
    while state==False:
        try:
            #PROXY = proxies[count].get_address()
            PROXY = proxies_address[count]
            print(PROXY)
            webdriver.DesiredCapabilities.CHROME['proxy']={
                                                            "httpProxy":PROXY,
                                                           "ftpProxy":PROXY,
                                                           "sslProxy":PROXY,
                                                           "proxyType":"MANUAL",
                                                            }
            driver = webdriver.Chrome(chromedriver)
            driver.get(listing_urls_list[i])

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            listing_data = get_data(soup)
            
            #try:
            #    check = (soup.find('span', itemprop="streetAddress")).text
            #except:
            #    bad_url.append(listing_urls_list[i])
            #    break
      
                
            
            tested_proxies.append(PROXY)
            state = True
        except:
            bad_proxies.append(PROXY)
            count += 1
            driver.close()
        
    driver.close()
    
    #insert listing data into mongo database
    col.insert_one(listing_data)
    print(listing_data)
    count += 1
    print(count)
    
    
    
    

    
    #if i//5 == 0:
    #    sleep(random.uniform(60,120))
        
    ####     
    #session=requests.Session()
    #session.headers = user_agent#{'User-Agent': ua.random} #'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    #print(user_agent)
    ##base_url='https://www.realtor.com'
    ##re_list=[]
    ##realtor_default_count=44
    ##page_counter=1
    ##search_url=''.join([base_url, '/realestateandhomes-search/Sacramento_CA'])
    #page=session.get(listing_urls_list[i])
    #c=page.content
    #soup=BeautifulSoup(c,"html.parser")
    ####
    
    #options = Options()
    #ua = UserAgent()
    #userAgent = ua.random
    #print(userAgent)
    #options.add_argument(f'user-agent={userAgent}')
    #driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    #driver.get(listing_urls_list[i])
    
    #sleep(random.uniform(120,180)) #pause

In [ ]:
tested_proxies

In [ ]:
get_data(soup)

In [ ]:
tested_proxy

In [ ]:
hello = {'hi':[]}

In [ ]:
hello['hi'] == []

In [ ]:
import re

In [ ]:
get_data(soup)

In [ ]:
bad_proxies

In [ ]:
for i in range(listing_from, listing_to+1): 
    
    print(i, listing_urls_list[i])
    
    #ua = UserAgent()
    #user_agent = {'User-agent': ua.random}
    #print(user_agent)

    #response  = requests.get(listing_urls_list[i], headers = user_agent)
    
    #driver = webdriver.Chrome(chromedriver)
    #driver.get(listing_urls_list[i])
    
    #if i//5 == 0:
    #    sleep(random.uniform(60,120))
        
    ####     
    #session=requests.Session()
    #session.headers = user_agent#{'User-Agent': ua.random} #'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    #print(user_agent)
    ##base_url='https://www.realtor.com'
    ##re_list=[]
    ##realtor_default_count=44
    ##page_counter=1
    ##search_url=''.join([base_url, '/realestateandhomes-search/Sacramento_CA'])
    #page=session.get(listing_urls_list[i])
    #c=page.content
    #soup=BeautifulSoup(c,"html.parser")
    ####
    
    #options = Options()
    #ua = UserAgent()
    #userAgent = ua.random
    #print(userAgent)
    #options.add_argument(f'user-agent={userAgent}')
    #driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    #driver.get(listing_urls_list[i])

    ###
    
    '''options = Options()
    ua = UserAgent()
    userAgent = ua.random
    #print(userAgent)
    options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    driver.get(listing_urls_list[i])
    #user_agent = {'User-agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}
    #response  = requests.get(listing_urls_list[i], headers = user_agent)
    
    sleep(random.uniform(60,100))'''
    ###
    
   
    
    PROXY = proxies[count].get_address()
    
    
    print(PROXY)
    webdriver.DesiredCapabilities.CHROME['proxy']={
                                                    "httpProxy":PROXY,
                                                    "ftpProxy":PROXY,
                                                    "sslProxy":PROXY,

                                                    "proxyType":"MANUAL",

                                                    }
    driver = webdriver.Chrome(chromedriver)
    driver.get(listing_urls_list[i])
    
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    

    driver.close()
    
    #insert listing data into mongo database
    col.insert_one(listing_data)
    print(listing_data)
    count += 1
    
    #sleep(random.uniform(120,180)) #pause

In [ ]:
proxies

In [ ]:
listing_data

In [ ]:
test = ('hi',50)

In [ ]:
test[1]

In [ ]:
type(test)

In [ ]:
def test(num):
    return num+12, num-12

In [ ]:
plus, minus = test(12)

In [ ]:
plus

In [ ]:
minus

In [ ]:
proxies[1].get_address()

In [ ]:
proxies

In [ ]:
PROXY = proxies[0].get_address()

In [ ]:
PROXY

In [ ]:
webdriver.DesiredCapabilities.CHROME['proxy']={
                                                            "httpProxy":PROXY,
                                                           "ftpProxy":PROXY,
                                                           "sslProxy":PROXY,
                                                           "proxyType":"MANUAL",
                                                            }

In [ ]:
driver = webdriver.Chrome(chromedriver)
driver.get(listing_urls_list[0])

In [ ]:
listing_urls_list[0]

In [ ]:
type(listing_urls_list)

In [ ]:
listing_urls_list[:20]

In [ ]:
soup

In [ ]:
try:
    check = (soup.find('span', itemprop="streetAddress")).text
except:

In [ ]:
proxies_address

In [ ]:
street_address = ('Not Found')
test = {'street_address': street_address}

In [ ]:
listing_data['street_address'] == 'Not Found'

In [ ]:
type(soup)

In [ ]:
client = MongoClient()

In [ ]:
client.list_database_names()

In [ ]:
db = client.get_database('philadelphia_home_prices')

In [ ]:
col = db.get_collection('listing_data')

In [ ]:
col.count()

In [ ]:
cursor = db.listing_data.find({})
listings = list(cursor)

In [ ]:
len(listings)

In [ ]:
listings

In [ ]:
import get_proxies as proxies
from extractors import get_data 

In [ ]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.realtor.com/realestateandhomes-detail/4506-Locust-St_Philadelphia_PA_19139_M33501-78957")

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
listing_data = get_data(soup)
print(listing_data)

In [ ]:
soup_string = str(soup)
soup_string